# Real-Encoded Genetic Algorithm

## 1. Problem Definition

Let considering the optimization problem:

$$\min_{x \in \mathbb{R}}{f(x)}=100 \times (x_{1}^{2} - x_{2})+(1-x_{1})^{2}$$

subject to following bounds and constraints:
$$0 \leq x_{1},x_{2} \leq 5$$

## 2. Implementation

In this assingment, we will try to implement a Real-value coded Genetic Algorithm from scratch to solve the above optimization problem.

We first import the necessary libraries

In [4]:
import numpy as np
np.random.seed(42)

We also define the initial fitness value for this optimization problem

In [5]:
MIN_VALUE = 9999

Moreover, we also make sure the bounding constraint for $x_1$ and $x_2$.

In [12]:
def check_constraints(x: list):
    """
    Check constraints of x1 and x2
    ---
    TODO: return True if x1 and x2 sastify the contraints, otherwise, return False  
    """
    # write your code below
    if 0<=x[0]<=5 and 0<=x[1]<=5:
        return True
    else:
        return False
    

Then, we initial the population by `init_population()`. The population will be a list that contains possible solutions. A particular solution is defined as a list of two element.

In [20]:
def init_population(num_pop: int):
    """
    Initial the population
    ---
    TODO: Return a population of solutions
    The population will be a list that contains possible solutions. 
    A particular solution is defined as a list of two element.
    Hint: use np.random.uniform(low, high) to random x1, x2 in bounding range [low, high)
    """
    # write your code below
    pop_list = list()
    for i in range(num_pop):
        pop_list.append(np.random.uniform(low=0,high=5+1e-8,size=2).tolist())
        
    return pop_list
    

In [21]:
# test the function
pop = init_population(6)
pop

[[3.03772426558264, 0.8526206201416989],
 [0.3252579655769135, 4.744427695755522],
 [4.828160175029117, 4.041986748666279],
 [1.523068848912991, 0.4883605710086405],
 [3.4211651394031146, 2.200762473099531],
 [0.6101911754442765, 2.47588455550812]]

We define `fitness()` function that take a list of two elements as input, and return the fitness score.

In [24]:
def fitness(x: list):
    """
    Fitness function
    ---
    TODO: return the fitness score
    Hint: use define of f(x)
    """
    # write your code below
    fitness_score = 100*(x[0]**2-x[1])+(1-x[0])**2
    return fitness_score
    

In [27]:
# test the function
# pop[0]
fitness(pop[0])

841.6671295393334

Like the tutorial, we also use the roulette selection method to choose parents.

In [28]:
def roulette_selection(pop: list, scores: list, n: int):
    # compute the sum of fitness
    fitness_total = float(sum(scores))
    # compute probability of fitness
    fitness_prob = [f / fitness_total for f in scores]
    # compute cumulative probability of fitness
    fitness_cum_prob = [sum(fitness_prob[: i + 1]) for i in range(len(fitness_prob))]
    # list of selected parents
    pop_selected = []
    for _ in range(n):
        r = np.random.uniform()
        for i, p in enumerate(pop):
            if r <= fitness_cum_prob[i]:
                pop_selected.append(p)
                break
    return pop_selected

For reproduction step, we apply the **blend crossover** function. Beside crossover rate `pc`, the fucntion has one more hyperparameter $\alpha \in [0,1] \in \mathbb{R}$ to compute blending weight.

In [35]:
def crossover(p1: list, p2: list, pc: float, alpha: float):
    """
    Blend crossover function for real-coded GA
    ---
    TODO: if the random value r smaller than pc, do the blend crossover as describe in the lecture slide
    """
    # children are copies of parents by default
    c1, c2 = p1.copy(), p2.copy()
    # check for recombination
    r = np.random.rand() 
    # write the code below
    if r < pc:
        w = (1+2*alpha)*r-alpha
        c1 = [(1-w)*p1[0]+w*p2[0],(1-w)*p1[1]+w*p2[1]]
        c2 = [(1-w)*p2[0]+w*p1[0],(1-w)*p2[1]+w*p1[1]]
    
    return c1, c2

In [38]:
# test the function
#print("before",c1,c2)
c1, c2 = crossover(pop[0], pop[1], 0.6, 0.5)
print(c1)
print(c2)

[1.5726234532745509, 2.9547258915753547]
[1.790358777885003, 2.6423224243218657]


For the `mutation()` function is **random mutation** as described in lecture slide with one more hyperparameter **delta**.

In [39]:
def mutation(p1: list, pm: float, delta: float):
    """
    Random mutation function for real-encoded GA
    ---
    TODO: if the random value r smaller than pm, do the random mutation as describe in the lecture slide. 
    NOTICE: before return the mutation result, please check the constraints
    If the mutation result is not sastify the constraint, return original input p1
    """
    c1 = p1.copy()
    for i in range(len(c1)):
        r = np.random.rand() 
        # write your code here
        if r < pm:
            c1[i] = c1[i]+(r-0.5)*delta
        
    # check the constraints
    # if c1 meets the constraints, return c1, else return p1
    # write your code below
    if check_constraints(c1):
        return c1
    else:
        return p1
    

In [40]:
# test the function
mutation(pop[0], 0.5, 2)

[3.03772426558264, 0.222329531192753]

Now, we can tie all of this together into a function named `genetic_algorithm()` that takes the name of the objective function and the hyperparameters of the search, and returns the best solution found during the search.

In [47]:
def genetic_algorithm(fitness_func: callable, num_iters: int, num_pop: int, pc: float, pm: float, alpha: float, delta: float):
    """
    @param fitness_func: the fitness function
    @param num_iters: the number of generation
    @param num_pop: the number of population
    @param pc: the crossover probability
    @param pm: the mutation probability
    @param alpha: the hyperparameter for blend crossover function
    @param delta: the hyperparameter for random mutation function
    """
    # initial population of random bitstring with corresponding fitness score
    # write your code below
    
    pop = init_population(num_pop)
    
    # keep track of best solution
    best, best_eval = pop[0], MIN_VALUE
    
    # enumerate generations
    for it in range(num_iters):
        # evaluate all candidates in the population
        scores = [fitness_func(c) for c in pop]
        
        # check for new best solution
        for i in range(num_pop):
            # wrtie your code below
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
            
        # select parents
        selected = roulette_selection(pop, scores, num_pop)

        # create the next generation
        children = []
        for i in range(0, num_pop, 2):
            # get selected parents in pairs
            # write your code below
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            # write your code below
            for c in crossover(p1, p2, pc, alpha):
                mutation(c, pm, delta)
                children.append(c)
            
        # replace old population with new one that having high scores
        pop = children
    return best, best_eval

Now, try the test case.

In [48]:
# define number of generations for finding the solution
n_iters = 10
# define the population size
n_pop = 6
# define crossover rate
pc = 0.5
# define mutation rate
pm = 0.001
#
alpha = 0.25
# 
delta = 2.5

In [49]:
# perform the genetic algorithm search
best, score = genetic_algorithm(fitness, n_iters, n_pop, pc, pm, alpha, delta)
print('Done!')
print(f'[x1, x2]: {best}, f([x1, x2]) = {score}')

Done!
[x1, x2]: [0.597971230887451, 3.5662239432474228], f([x1, x2]) = -320.7038078966428


Now, setup your own hyperpamameters to get the possible results you can obtain

In [67]:
# TODO: define hyperparameters by yourself to get the best result
# Write your code from here

# increase population size & number of generations
n_iters = 50
n_pop = 20
best, score = genetic_algorithm(fitness, n_iters, n_pop, pc, pm, alpha, delta)
print('Done!')
print(f'[x1, x2]: {best}, f([x1, x2]) = {score}')


Done!
[x1, x2]: [0.8050131254356643, 4.992883613894289], f([x1, x2]) = -434.4457282958069


In [73]:
n_iters = 100
n_pop = 100
best, score = genetic_algorithm(fitness, n_iters, n_pop, pc, pm, alpha, delta)
print('Done!')
print(f'[x1, x2]: {best}, f([x1, x2]) = {score}')

Done!
[x1, x2]: [0.3557702502372908, 4.973227697909967], f([x1, x2]) = -484.2504907251269


In [77]:
pm = 0.01
n_iters = 100
n_pop = 100
best, score = genetic_algorithm(fitness, n_iters, n_pop, pc, pm, alpha, delta)
print('Done!')
print(f'[x1, x2]: {best}, f([x1, x2]) = {score}')


Done!
[x1, x2]: [0.013686906152519815, 4.919594259844008], f([x1, x2]) = -490.967879325303


In [83]:
def rank_based_selection(pop: list, scores: list, n: int):
    fitness_total = float(sum(scores))
    fitness_prob = [f / fitness_total for f in scores]
    num = len(fitness_prob)
    rank_sum = num*(num+1)/2
    # re-arrange population list using rank order
    sorted_index = np.argsort(fitness_prob)
    pop = [pop[i] for i in sorted_index]

    new_fitness_prob = [i / rank_sum for i in range(1,num+1)]

    
    fitness_cum_prob = [sum(new_fitness_prob[: i + 1]) for i in range(len(new_fitness_prob))]
    # list of selected parents
    pop_selected = []
    for _ in range(n):
        r = np.random.uniform()
        for i, p in enumerate(pop):
            if r <= fitness_cum_prob[i]:
                pop_selected.append(p)
                break
    return pop_selected

In [84]:
def new_genetic_algorithm(fitness_func: callable, num_iters: int, num_pop: int, pc: float, pm: float, alpha: float, delta: float):
    """
    @param fitness_func: the fitness function
    @param num_iters: the number of generation
    @param num_pop: the number of population
    @param pc: the crossover probability
    @param pm: the mutation probability
    @param alpha: the hyperparameter for blend crossover function
    @param delta: the hyperparameter for random mutation function
    """
    # initial population of random bitstring with corresponding fitness score
    # write your code below
    
    pop = init_population(num_pop)
    
    # keep track of best solution
    best, best_eval = pop[0], MIN_VALUE
    
    # enumerate generations
    for it in range(num_iters):
        # evaluate all candidates in the population
        scores = [fitness_func(c) for c in pop]
        
        # check for new best solution
        for i in range(num_pop):
            # wrtie your code below
            if scores[i] < best_eval:
                best, best_eval = pop[i], scores[i]
            
        # select parents
        selected = rank_based_selection(pop, scores, num_pop)

        # create the next generation
        children = []
        for i in range(0, num_pop, 2):
            # get selected parents in pairs
            # write your code below
            p1, p2 = selected[i], selected[i+1]
            # crossover and mutation
            # write your code below
            for c in crossover(p1, p2, pc, alpha):
                mutation(c, pm, delta)
                children.append(c)
            
        # replace old population with new one that having high scores
        pop = children
    return best, best_eval

In [94]:
pm = 0.001
n_iters = 100
n_pop = 100
best, score = new_genetic_algorithm(fitness, n_iters, n_pop, pc, pm, alpha, delta)
print('Done!')
print(f'[x1, x2]: {best}, f([x1, x2]) = {score}')

Done!
[x1, x2]: [0.02031677070504383, 4.993930263021962], f([x1, x2]) = -498.3919699552462
